In [1]:
pip install folium pandas



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import folium

# Load your dataset (Replace 'your_file.csv' with the actual file path)
df = pd.read_csv("geo_CBP_apprehensions.csv")

# Create a base map centered at a general location (Mexico)
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

# Loop through DataFrame and add markers
for index, row in df.iterrows():
    lat, lon = row['lat'], row['lon']
    city, state, country = row['city_birth'], row['state_birth'], row['country_birth']
    
    if pd.notna(lat) and pd.notna(lon):  # Ensure lat/lon are not NaN
        popup_text = f"{city}, {state}, {country}"
        folium.Marker(
            location=[lat, lon],
            popup=popup_text,
            tooltip=popup_text
        ).add_to(m)

# Save and display the map
m.save("map.html")
m


In [2]:
import pandas as pd

# Load the dataset
file_path = "geo_CBP_apprehensions.csv"  # Replace with the path to your uploaded file
df = pd.read_csv(file_path, low_memory=False)

# Extract numeric fiscal year from the 'FY' column
df['FY'] = df['FY'].str.extract(r'(\d{4})').astype(float)

# Filter data for FY 2019 and FY 2022
df_filtered = df[df['FY'].isin([2019, 2022])]

# Group by relevant columns and count rows (each row represents one person)
df_grouped = df_filtered.groupby(['city_birth', 'country_birth', 'FY']).size().reset_index(name='Count')

# Check the grouped data
print("Grouped Data:")
print(df_grouped)

# Pivot the data to have FYs as columns
df_pivot = df_grouped.pivot(index=['city_birth', 'country_birth'], columns='FY', values='Count').fillna(0)

# Ensure 2019 and 2022 columns exist
if 2019 in df_pivot.columns and 2022 in df_pivot.columns:
    # Rename columns for clarity
    df_pivot.columns = ['2019', '2022']

    # Calculate Growth
    df_pivot['Growth'] = ((df_pivot['2022'] - df_pivot['2019']) / df_pivot['2019'].replace(0, None)) * 100

    # Handle Adjusted Growth for 2019 = 0
    df_pivot['Adjusted Growth'] = df_pivot['Growth']
    df_pivot.loc[df_pivot['2019'] == 0, 'Adjusted Growth'] = df_pivot['2022']

    # Replace NaN or infinite values with 0
    df_pivot['Growth'] = df_pivot['Growth'].fillna(0)
    df_pivot['Adjusted Growth'] = df_pivot['Adjusted Growth'].fillna(0)

    # Sort by Adjusted Growth
    df_sorted = df_pivot.sort_values(by='Adjusted Growth', ascending=False)

    # Save the sorted DataFrame to a new CSV file
    output_file = "sorted_by_adjusted_growth.csv"
    df_sorted.to_csv(output_file)

    print(f"Sorted and recalculated dataset saved to '{output_file}'.")
else:
    print("Error: FY 2019 or FY 2022 is missing in the dataset.")


Grouped Data:
                    city_birth country_birth      FY  Count
0      "SUBJECT ILLEGALLY ENTE      HONDURAS  2019.0      1
1          #1 CANTON NATIVIDAD   EL SALVADOR  2019.0      1
2                    (UNKNOWN)        MEXICO  2019.0      1
3                            ,     GUATEMALA  2019.0      2
4                            ,        MEXICO  2019.0      1
...                        ...           ...     ...    ...
72440                tlaquilpa        MEXICO  2019.0      1
72441                  unknown     NICARAGUA  2022.0      1
72442               villanueva     NICARAGUA  2019.0      1
72443                   xalapa        MEXICO  2019.0      1
72444    zihuatanejo de azueta        MEXICO  2019.0      1

[72445 rows x 4 columns]
Sorted and recalculated dataset saved to 'sorted_by_adjusted_growth.csv'.


In [2]:
import pandas as pd

# Load the dataset
file_path = "geo_CBP_apprehensions.csv"  # Replace with the path to your uploaded file
df = pd.read_csv(file_path, low_memory=False)

# Extract numeric fiscal year from the 'FY' column
df['FY'] = df['FY'].str.extract(r'(\d{4})').astype(float)

# Filter data for FY 2019 and FY 2022
df_filtered = df[df['FY'].isin([2019, 2022])]

# Group by relevant columns and count rows (each row represents one person)
df_grouped = df_filtered.groupby(['city_birth', 'country_birth', 'FY']).size().reset_index(name='Count')

# Pivot the data to have FYs as columns
df_pivot = df_grouped.pivot(index=['city_birth', 'country_birth'], columns='FY', values='Count').fillna(0)

# Ensure 2019 and 2022 columns exist
if 2019 in df_pivot.columns and 2022 in df_pivot.columns:
    # Rename columns for clarity
    df_pivot.columns = ['2019', '2022']

    # Calculate Growth
    df_pivot['Growth'] = ((df_pivot['2022'] - df_pivot['2019']) / df_pivot['2019'].replace(0, None)) * 100

    # Handle Adjusted Growth for 2019 = 0
    df_pivot['Adjusted Growth'] = df_pivot['Growth']
    df_pivot.loc[df_pivot['2019'] == 0, 'Adjusted Growth'] = df_pivot['2022']

    # Replace NaN or infinite values with 0
    df_pivot['Growth'] = df_pivot['Growth'].fillna(0)
    df_pivot['Adjusted Growth'] = df_pivot['Adjusted Growth'].fillna(0)

    # Round Growth and Adjusted Growth to 2 decimal places
    df_pivot['Growth'] = df_pivot['Growth'].round(2)
    df_pivot['Adjusted Growth'] = df_pivot['Adjusted Growth'].round(2)

    # Sort by Adjusted Growth
    df_sorted = df_pivot.sort_values(by='Adjusted Growth', ascending=False)

    # Save the sorted DataFrame to a new CSV file
    output_file = "sorted_by_adjusted_growth.csv"
    df_sorted.to_csv(output_file)

    print(f"Sorted and recalculated dataset saved to '{output_file}'.")
else:
    print("Error: FY 2019 or FY 2022 is missing in the dataset.")


KeyboardInterrupt: 

In [4]:
import pandas as pd
import folium
from folium.plugins import TimestampedGeoJson

# Load dataset (Replace with actual file path)
file_path = "geo_CBP_apprehensions.csv"
df = pd.read_csv(file_path, low_memory=False)

# Extract numeric fiscal year (e.g., "FY2010" → 2010)
df['FY'] = df['FY'].str.extract(r'(\d{4})').astype(int)

# Filter for Mexico
df_mexico = df[df['country_birth'].str.upper() == "BELIZE"]

# Group by city and year to count people per city per year
df_grouped = df_mexico.groupby(['city_birth', 'state_birth', 'lat', 'lon', 'FY']).size().reset_index(name='count')

# Create a base map centered on Mexico
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

# Prepare data for TimestampedGeoJson
features = []
for _, row in df_grouped.iterrows():
    lat, lon = row['lat'], row['lon']
    city, state, year, count = row['city_birth'], row['state_birth'], row['FY'], row['count']
    
    if pd.notna(lat) and pd.notna(lon):  # Ensure lat/lon are valid
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [lon, lat],  # GeoJSON format: [longitude, latitude]
            },
            "properties": {
                "time": f"{year}-01-01",  # Use a consistent date format per year
                "popup": f"<b>City:</b> {city}<br><b>State:</b> {state}<br><b>Year:</b> {year}<br><b>People:</b> {count}",
                "icon": "circle",
                "iconstyle": {
                    "color": "red",
                    "fillColor": "red",
                    "fillOpacity": 0.6,
                    "radius": (count ** 0.5) * 2  # Adjust size dynamically
                },
            },
        }
        features.append(feature)

# Add Timestamped GeoJson (Animated without visible timeline bar)
TimestampedGeoJson(
    {"type": "FeatureCollection", "features": features},
    period="P1Y",  # One-year intervals
    add_last_point=True,
    auto_play=True,  # Automatically play animation
    loop=True,  # Allow looping
    max_speed=1,  # 1 FPS animation speed
    loop_button=False,  # Hide loop button
    date_options="YYYY",
    time_slider_drag_update=False,  # Hide the time slider
).add_to(m)

# Save and display map
output_file = "belize.html"
m.save(output_file)

print(f"Map has been saved as '{output_file}'. Open it in your browser to explore.")


Map has been saved as 'belize.html'. Open it in your browser to explore.


In [5]:
import pandas as pd
import folium
from folium.plugins import TimestampedGeoJson

# Load dataset (Replace with actual file path)
file_path = "geo_CBP_apprehensions.csv"
df = pd.read_csv(file_path, low_memory=False)

# Extract numeric fiscal year (e.g., "FY2010" → 2010)
df['FY'] = df['FY'].str.extract(r'(\d{4})').astype(int)

# Filter for Mexico
df_mexico = df[df['country_birth'].str.upper() == "BELIZE"]

# Group by city and year to count people per city per year
df_grouped = df_mexico.groupby(['city_birth', 'state_birth', 'lat', 'lon', 'FY']).size().reset_index(name='count')

# Create a base map centered on Mexico
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

# Prepare data for TimestampedGeoJson
features = []
for _, row in df_grouped.iterrows():
    lat, lon = row['lat'], row['lon']
    city, state, year, count = row['city_birth'], row['state_birth'], row['FY'], row['count']
    
    if pd.notna(lat) and pd.notna(lon):  # Ensure lat/lon are valid
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [lon, lat],  # GeoJSON format: [longitude, latitude]
            },
            "properties": {
                "time": f"{year}-01-01",  # Use a consistent date format per year
                "popup": f"<b>City:</b> {city}<br><b>State:</b> {state}<br><b>Year:</b> {year}<br><b>People:</b> {count}",
                "icon": "circle",
                "iconstyle": {
                    "color": "red",
                    "fillColor": "red",
                    "fillOpacity": 0.6,
                    "radius": (count ** 0.5) * 2  # Adjust size dynamically
                },
            },
        }
        features.append(feature)

# Add Timestamped GeoJson (for animated year slider)
TimestampedGeoJson(
    {"type": "FeatureCollection", "features": features},
    period="P1Y",  # One-year intervals
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options="YYYY",
    time_slider_drag_update=True,
).add_to(m)

# Save and display map
output_file = "belize.html"
m.save(output_file)

print(f"Map has been saved as '{output_file}'. Open it in your browser to explore.")


Map has been saved as 'belize.html'. Open it in your browser to explore.


In [6]:
import pandas as pd
import folium
from folium.plugins import TimestampedGeoJson

# Load dataset (Replace with actual file path)
file_path = "geo_CBP_apprehensions.csv"
df = pd.read_csv(file_path, low_memory=False)

# Extract numeric fiscal year (e.g., "FY2010" → 2010)
df['FY'] = df['FY'].str.extract(r'(\d{4})').astype(int)

# Filter for Belize
df_belize = df[df['country_birth'].str.upper() == "BELIZE"]

# Group by city and year to count people per city per year
df_grouped = df_belize.groupby(['city_birth', 'state_birth', 'lat', 'lon', 'FY']).size().reset_index(name='count')

# Create a base map centered on Belize
m = folium.Map(location=[17.1899, -88.4976], zoom_start=7)  # Belize's coordinates

# Prepare data for TimestampedGeoJson
features = []
for _, row in df_grouped.iterrows():
    lat, lon = row['lat'], row['lon']
    city, state, year, count = row['city_birth'], row['state_birth'], row['FY'], row['count']
    
    if pd.notna(lat) and pd.notna(lon):  # Ensure lat/lon are valid
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [lon, lat],  # GeoJSON format: [longitude, latitude]
            },
            "properties": {
                "time": f"{year}-01-01",  # Use a consistent date format per year
                "popup": f"<b>City:</b> {city}<br><b>State:</b> {state}<br><b>Year:</b> {year}<br><b>People:</b> {count}",
                "icon": "circle",
                "iconstyle": {
                    "color": "red",
                    "fillColor": "red",
                    "fillOpacity": 0.6,
                    "radius": (count ** 0.5) * 2  # Adjust size dynamically
                },
            },
        }
        features.append(feature)

# Add Timestamped GeoJson (for animated year slider)
TimestampedGeoJson(
    {"type": "FeatureCollection", "features": features},
    period="P1Y",  # One-year intervals
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options="YYYY",
    time_slider_drag_update=True,
).add_to(m)

# Save and display map
output_file = "belize.html"
m.save(output_file)

print(f"Map has been saved as '{output_file}'. Open it in your browser to explore.")


Map has been saved as 'belize.html'. Open it in your browser to explore.


In [7]:
import pandas as pd
import folium
from folium.plugins import TimestampedGeoJson

# Load dataset (Replace with actual file path)
file_path = "geo_CBP_apprehensions.csv"
df = pd.read_csv(file_path, low_memory=False)

# Extract numeric fiscal year (e.g., "FY2010" → 2010)
df['FY'] = df['FY'].str.extract(r'(\d{4})').astype(int)

# Filter for Belize
df_belize = df[df['country_birth'].str.upper() == "BELIZE"]

# Group by city and year to count people per city per year
df_grouped = df_belize.groupby(['city_birth', 'state_birth', 'lat', 'lon', 'FY']).size().reset_index(name='count')

# Create a base map (zoom will be adjusted later)
m = folium.Map(zoom_start=7)

# Prepare data for TimestampedGeoJson
features = []
for _, row in df_grouped.iterrows():
    lat, lon = row['lat'], row['lon']
    city, state, year, count = row['city_birth'], row['state_birth'], row['FY'], row['count']
    
    if pd.notna(lat) and pd.notna(lon):  # Ensure lat/lon are valid
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [lon, lat],  # GeoJSON format: [longitude, latitude]
            },
            "properties": {
                "time": f"{year}-01-01",  # Use a consistent date format per year
                "popup": f"<b>City:</b> {city}<br><b>State:</b> {state}<br><b>Year:</b> {year}<br><b>People:</b> {count}",
                "icon": "circle",
                "iconstyle": {
                    "color": "red",
                    "fillColor": "red",
                    "fillOpacity": 0.6,
                    "radius": (count ** 0.5) * 2  # Adjust size dynamically
                },
            },
        }
        features.append(feature)

# Add Timestamped GeoJson (for animated year slider)
TimestampedGeoJson(
    {"type": "FeatureCollection", "features": features},
    period="P1Y",  # One-year intervals
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options="YYYY",
    time_slider_drag_update=True,
).add_to(m)

# Auto-zoom to fit all points in Belize
bounds = [[row['lat'], row['lon']] for _, row in df_grouped.iterrows() if pd.notna(row['lat']) and pd.notna(row['lon'])]
if bounds:
    m.fit_bounds(bounds)

# Save and display map
output_file = "belize.html"
m.save(output_file)

print(f"Map has been saved as '{output_file}'. Open it in your browser to explore.")


Map has been saved as 'belize.html'. Open it in your browser to explore.
